### 페어 RDD 생성
파이썬에서는 키를 가지는 데이터로 동작하는 함수들을 고려해 튜플로 구성된 RDD를 되돌려 주어야한다.
```
pairs = lines.map(lambda x : (x.split(" ")[0], x))
```
### 페어 RDD의 트랜스포메이션
1. reduceByKey(func)  
동일한 키에 대한 값들을 합친다.
```
lines = sc.parallelize([(1,2), (3,4), (3,6)])
lines.reduceByKey(lambda x, y : x+y).collect()
=> [(1, 2), (3, 10)]
```
2. groupByKey  
동일 키에 대한 값들을 그룹화한다.
```
lines = sc.parallelize([(1,2), (3,4), (3,6)])
lines = lines.groupByKey()
for key, value in lines.collect():
    print(key, list(value))
=> 1 [2], 3 [4, 6]
```
3. mapValues(func)
키의 변경 없이 페어 RDD의 각 값에 함수를 적용한다.
```
lines = sc.parallelize([(1,2), (3,4), (3,6)])
lines.mapValues(lambda x : x+1).collect()
=> [(1, 3), (3, 5), (3, 7)]
```
4. keys()
RDD가 가진 키들만을 되돌려 준다.
```
lines = sc.parallelize([(1,2), (3,4), (3,6)])
lines.keys().collect()
=> [1,3,3]
```
5. values()
RDD가 가진 값들만을 되돌려 준다.
```
lines = sc.parallelize([(1,2), (3,4), (3,6)])
lines.values().collect()
=>[2,4,6]
```
6. sortByKey()
키로 정렬된 RDD를 돌려준다.
```
lines = sc.parallelize([(1,2), (3,4), (3,6)])
lines.sortByKey().collect()
=>[(1,2),(3,4),(3,6)]
```

### 두 페어 RDD에 대한 트랜스포메이션
1. subtractByKey()
키를 기준으로 제거한다.
```
rdd = sc.parallelize([(1,2), (3,4), (3,6)])
other = sc.parallelize([(3,9)])
rdd.subtractByKey(other).collect()
=> [(1,2)]
```
2. join
키를 기준으로 inner join한다.
```
rdd = sc.parallelize([(1,2), (3,4), (3,6)])
other = sc.parallelize([(3,9)])
rdd.join(other).collect()
=> [(3, (4, 9)), (3, (6, 9))]
```
3. leftOuterJoin
```
rdd = sc.parallelize([(1,2), (3,4), (3,6)])
other = sc.parallelize([(3,9)])
rdd.leftOuterJoin(other).collect()
=> [(1, (2, None)), (3, (4, 9)), (3, (6, 9))]
```
4. cogroup
동일 키에 대해 양쪽 RDD를 그룹화한다.
```
rdd = sc.parallelize([(1,2), (3,4), (3,6)])
other = sc.parallelize([(3,9)])
for key, vaule in rdd.cogroup(other).collect():
    print(key, list(value), end=" ")
=> [(1, ([2],[])), (3, ([4,6],[9]))]
```

## 집합 연산
### reduceByKey()
reduceByKey()는 reduce()와 매우 유사하다. 둘 다 함수를 받아 값들을 합치는 데에 사용한다.
- 키 별 평균하기
```
rdd = sc.parallelize([('panda',0), ('pink',3), ('pirate',3), ('panda',1), ('pink',4)])
rdd.mapValues(lambda x : (x,1)).reduceByKey(lambda x, y : (x[0]+y[0], x[1]+y[1])).collect()
=> [('panda', (1, 2)), ('pink', (7, 2)), ('pirate', (3, 1))]
```
### combineByKey()
combineByKey()는 키별 집합 연산 중 가장 일반적으로 쓰인다.  
aggregate()와 마찬가지로 combineByKey() 또한 입력 데이터와 동일한 타입의 값을 되돌려 줄 필요가 없다.  
combineByKey()는 한 파티션 내의 데이터들을 하나 씩 처리하게 되며, 각 데이터는 이전에 나온 적이 없는 키를 갖고 있을 수도 있고 같은 키도 갖을 수 있다.  